In [10]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
import torch


In [12]:
# Load tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


In [14]:
# Load synthetic dataset (adjust path if needed)
df = pd.read_csv("synthetic_radiology_reports.csv")

# View first few rows
df.head()



,report,summary
0,There is a right lower lobe opacity with air b...,Opacity in right lower lobe.
1,No consolidation is seen in the cardiac silhou...,cardiac silhouette consolidation.
2,Clear lungs. nodule present in the cardiac sil...,"Normal lungs, but nodule seen in cardiac silho..."
3,Clear lungs. consolidation present in the left...,"Normal lungs, but consolidation seen in left u..."
4,Nodule noted in the right lower lobe.,No nodule in the right lower lobe.


In [16]:
# Function to summarize one report
def summarize_text(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        length_penalty=2.0,
        max_length=100,
        min_length=20,
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Only run on first N rows to start (e.g. 100 for testing)
df_sample = df.head(100).copy()

# Generate summaries
df_sample['generated_summary'] = df_sample['report'].apply(summarize_text)


In [18]:
# Show a few results
df_sample[['report', 'summary', 'generated_summary']].head(10)


,report,summary,generated_summary
0,There is a right lower lobe opacity with air b...,Opacity in right lower lobe.,There is a right lower lobe opacity with air b...
1,No consolidation is seen in the cardiac silhou...,cardiac silhouette consolidation.,No consolidation is seen in the cardiac silhou...
2,Clear lungs. nodule present in the cardiac sil...,"Normal lungs, but nodule seen in cardiac silho...",Clear lungs. nodule present in the cardiac sil...
3,Clear lungs. consolidation present in the left...,"Normal lungs, but consolidation seen in left u...",Clear lungs. consolidation present in the left...
4,Nodule noted in the right lower lobe.,No nodule in the right lower lobe.,Nodule noted in the right lower lobe. Nodule a...
5,No pleural effusion is seen in the left upper ...,"Normal lungs, but pleural effusion seen in lef...",No pleural effusion is seen in the left upper ...
6,"Mild nodule detected in the mediastinum, consi...",Nodule in mediastinum.,"Mild nodule detected in the mediastinum, consi..."
7,Clear lungs. nodule present in the left upper ...,Mild nodule suggesting pneumonia.,Clear lungs. nodule present in the left upper ...
8,Clear lungs. opacity present in the right lowe...,Opacity in right lower lobe.,Clear lungs. opacity present in the right lowe...
9,No pleural effusion is seen in the left upper ...,"Normal lungs, but pleural effusion seen in lef...",No pleural effusion is seen in the left upper ...
